# Batch Telemetry Data Download

This notebook downloads telemetry data (JSON files) for all matches across multiple tournaments.

## Features
- Processes multiple tournaments from CSV files
- Downloads all match telemetry files
- Rate limiting (300 requests/min)
- Error handling for failed tournaments/matches
- Progress tracking


In [0]:
# Import required libraries
import requests
import json
import os
import time
import glob
import pandas as pd
from pathlib import Path

# ============================================================================
# CONFIGURATION
# ============================================================================
api_key = "{your_key}"
platform = "tournament"
dest_path = "{your_path}"
timeout = 15
rate_limit_delay = 0.2  # seconds between API calls (~300 requests/min)

# TEST MODE: Set to None to process all tournaments, or a number to limit for testing
test_limit = 1  # Process only first 3 tournaments for testing

# ============================================================================

# Create output directory if it doesn't exist
os.makedirs(dest_path, exist_ok=True)

print("✅ Setup complete!")
print(f"Configuration loaded:")
print(f"  • Output Directory: {dest_path}")
print(f"  • Platform: {platform}")
print(f"  • Rate Limit: {1/rate_limit_delay:.0f} requests/min")
if test_limit:
    print(f"  • TEST MODE: Processing only first {test_limit} tournaments")


In [0]:
# Find all CSV files in the subfolder "tournamentid"
# For Databricks: Use workspace path or relative path depending on where CSV files are uploaded
csv_files = glob.glob("tournamentid/*.csv")

# List to store all processed tournament IDs
all_tournament_ids = []

# Process each CSV file separately
for csv_file in csv_files:
    # Read the csv file
    df = pd.read_csv(csv_file)
    
    # Get the column name (should be only one column)
    column_name = df.columns[0]
    
    # Convert to string type and filter values starting with "es_"
    df[column_name] = df[column_name].astype(str)
    es_values = df[df[column_name].str.startswith("es_", na=False)][column_name]
    
    # Split by "_" and retain only the second element
    processed_values = es_values.str.split("_").str[1]
    
    # Add to the list
    all_tournament_ids.extend(processed_values.tolist())

# Remove duplicates and empty strings by converting to set and filtering
tournament_ids = [tid for tid in set(all_tournament_ids) if tid.strip()]

# Show how many values we have
print(f"✅ Total unique tournament IDs extracted: {len(tournament_ids)}")
print(f"\nTournament IDs: {tournament_ids}")


In [0]:
def download_telemetry(url, match_id, output_dir, verbose=False):
    """
    Download a telemetry file from PUBG CDN
    
    Args:
        url (str): URL to the telemetry file
        match_id (str): Match ID to use as filename
        output_dir (str): Directory to save the file
        verbose (bool): Print progress information (default: False)
    
    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Set up headers for the request
        headers = {
            'Accept': 'application/vnd.api+json',
            'Accept-Encoding': 'gzip, deflate, br',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        # Make the request with streaming support
        response = requests.get(url, headers=headers, stream=True, timeout=timeout)
        response.raise_for_status()
        
        # Full path to save file (using match_id as filename)
        filepath = os.path.join(output_dir, f"{match_id}.json")
        
        # Download and save the file
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        
        if verbose:
            file_size = os.path.getsize(filepath)
            if file_size < 1024 * 1024:
                size_str = f"{file_size / 1024:.2f} KB"
            else:
                size_str = f"{file_size / (1024 * 1024):.2f} MB"
            print(f"    ✅ Downloaded: {match_id}.json ({size_str})")
        
        return True
    
    except Exception as e:
        if verbose:
            print(f"    ❌ Failed: {match_id} - {str(e)}")
        return False

print("✅ Download function defined")


In [0]:
# Initialize counters
total_tournaments_processed = 0
total_tournaments_failed = 0
total_matches_downloaded = 0
total_matches_failed = 0

# Headers for API requests
api_headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/vnd.api+json"
}

# Apply test limit if set
tournaments_to_process = tournament_ids[:test_limit] if test_limit else tournament_ids

print(f"🚀 Starting batch download for {len(tournaments_to_process)} tournaments...\n")
print("="*70)

# Iterate through each tournament ID
for idx, tournament_id in enumerate(tournaments_to_process, 1):
    print(f"\n[{idx}/{len(tournaments_to_process)}] Processing tournament: {tournament_id}")
    
    try:
        # Fetch tournament data
        url = f"https://api.pubg.com/tournaments/{tournament_id}"
        response = requests.get(url, headers=api_headers, timeout=timeout)
        response.raise_for_status()
        tournament_data = response.json()
        
        # Apply rate limiting
        time.sleep(rate_limit_delay)
        
        # Extract match IDs
        matches = tournament_data['data']['relationships']['matches']['data']
        num_matches = len(matches)
        print(f"  Found {num_matches} matches")
        
        # Process each match
        matches_success = 0
        matches_fail = 0
        
        for match_idx, match_info in enumerate(matches, 1):
            match_id = match_info['id']
            
            try:
                # Fetch match details
                match_url = f"https://api.pubg.com/shards/{platform}/matches/{match_id}"
                match_response = requests.get(match_url, headers=api_headers, timeout=timeout)
                match_response.raise_for_status()
                match_data = match_response.json()
                
                # Apply rate limiting
                time.sleep(rate_limit_delay)
                
                # Extract telemetry URL
                asset_id = match_data["data"]["relationships"]["assets"]["data"][0]["id"]
                telemetry_url = None
                
                for asset in match_data.get("included", []):
                    if asset["type"] == "asset" and asset["id"] == asset_id:
                        telemetry_url = asset["attributes"]["URL"]
                        break
                
                if telemetry_url:
                    # Download telemetry file
                    success = download_telemetry(telemetry_url, match_id, dest_path, verbose=False)
                    
                    if success:
                        matches_success += 1
                        print(f"  [{match_idx}/{num_matches}] ✅ {match_id}")
                    else:
                        matches_fail += 1
                        print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - Download failed")
                else:
                    matches_fail += 1
                    print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - No telemetry URL")
                
            except Exception as e:
                matches_fail += 1
                print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - Error: {str(e)[:50]}")
                continue
        
        # Update tournament counters
        total_tournaments_processed += 1
        total_matches_downloaded += matches_success
        total_matches_failed += matches_fail
        
        print(f"  Tournament summary: {matches_success} downloaded, {matches_fail} failed")
        
    except Exception as e:
        total_tournaments_failed += 1
        print(f"  ❌ Failed to process tournament: {str(e)[:80]}")
        continue

# Print final summary
print("\n" + "="*70)
print("🎉 Batch download complete!\n")
print(f"Summary:")
print(f"  • Tournaments processed: {total_tournaments_processed}/{len(tournaments_to_process)}")
print(f"  • Tournaments failed: {total_tournaments_failed}")
print(f"  • Matches downloaded: {total_matches_downloaded}")
print(f"  • Matches failed: {total_matches_failed}")
print(f"  • Success rate: {total_matches_downloaded/(total_matches_downloaded+total_matches_failed)*100:.1f}%" if (total_matches_downloaded+total_matches_failed) > 0 else "  • Success rate: N/A")
print("="*70)


In [0]:
missed_tournament_ids = ["test-pnc25", "as-pnc25gs", "as-pnc25fs", "test-pgs7", "as-pgs7gs", "as-pgs7fs", "test-pgs8", "as-pgs8gs", "as-pgs8fs", "as-pgs9api", 
 "as-pgs9gs", "as-pgs9fs", "test-ewc25", "as-ewc25gs", "as-ewc25fs"]

# Iterate through each tournament ID
for idx, tournament_id in enumerate(missed_tournament_ids, 1):
    print(f"\n[{idx}/{len(missed_tournament_ids)}] Processing tournament: {tournament_id}")
    
    try:
        # Fetch tournament data
        url = f"https://api.pubg.com/tournaments/{tournament_id}"
        response = requests.get(url, headers=api_headers, timeout=timeout)
        response.raise_for_status()
        tournament_data = response.json()
        
        # Apply rate limiting
        time.sleep(rate_limit_delay)
        
        # Extract match IDs
        matches = tournament_data['data']['relationships']['matches']['data']
        num_matches = len(matches)
        print(f"  Found {num_matches} matches")
        
        # Process each match
        matches_success = 0
        matches_fail = 0
        
        for match_idx, match_info in enumerate(matches, 1):
            match_id = match_info['id']
            
            try:
                # Fetch match details
                match_url = f"https://api.pubg.com/shards/{platform}/matches/{match_id}"
                match_response = requests.get(match_url, headers=api_headers, timeout=timeout)
                match_response.raise_for_status()
                match_data = match_response.json()
                
                # Apply rate limiting
                time.sleep(rate_limit_delay)
                
                # Extract telemetry URL
                asset_id = match_data["data"]["relationships"]["assets"]["data"][0]["id"]
                telemetry_url = None
                
                for asset in match_data.get("included", []):
                    if asset["type"] == "asset" and asset["id"] == asset_id:
                        telemetry_url = asset["attributes"]["URL"]
                        break
                
                if telemetry_url:
                    # Download telemetry file
                    success = download_telemetry(telemetry_url, match_id, dest_path, verbose=False)
                    
                    if success:
                        matches_success += 1
                        print(f"  [{match_idx}/{num_matches}] ✅ {match_id}")
                    else:
                        matches_fail += 1
                        print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - Download failed")
                else:
                    matches_fail += 1
                    print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - No telemetry URL")
                
            except Exception as e:
                matches_fail += 1
                print(f"  [{match_idx}/{num_matches}] ❌ {match_id} - Error: {str(e)[:50]}")
                continue
        
        # Update tournament counters
        total_tournaments_processed += 1
        total_matches_downloaded += matches_success
        total_matches_failed += matches_fail
        
        print(f"  Tournament summary: {matches_success} downloaded, {matches_fail} failed")
        
    except Exception as e:
        total_tournaments_failed += 1
        print(f"  ❌ Failed to process tournament: {str(e)[:80]}")
        continue

# Print final summary
print("\n" + "="*70)
print("🎉 Batch download complete!\n")
print(f"Summary:")
print(f"  • Tournaments processed: {total_tournaments_processed}/{len(tournaments_to_process)}")
print(f"  • Tournaments failed: {total_tournaments_failed}")
print(f"  • Matches downloaded: {total_matches_downloaded}")
print(f"  • Matches failed: {total_matches_failed}")
print(f"  • Success rate: {total_matches_downloaded/(total_matches_downloaded+total_matches_failed)*100:.1f}%" if (total_matches_downloaded+total_matches_failed) > 0 else "  • Success rate: N/A")
print("="*70)


In [0]:
tournament_match_counts = {}

for tournament_id in tournament_ids:
    try:
        url = f"https://api.pubg.com/tournaments/{tournament_id}"
        response = requests.get(url, headers=api_headers, timeout=timeout)
        response.raise_for_status()
        tournament_data = response.json()
        matches = tournament_data['data']['relationships']['matches']['data']
        num_matches = len(matches)
        tournament_match_counts[tournament_id] = num_matches
        time.sleep(rate_limit_delay)
    except Exception as e:
        tournament_match_counts[tournament_id] = None

# Filter tournaments with more than 100 matches
large_tournaments = {tid: count for tid, count in tournament_match_counts.items() if count and count > 100}

for tid, count in large_tournaments.items():
    print(f"{tid} {count}")